In [1]:

from statsbombpy import sb 
from xgboost import XGBClassifier
import polars as pl
import mplsoccer as mpl

# Modelling pass difficulties
I believe we can all agree that Pavard's half volley goal was a beautiful, but I the buildup to the goal is also really interesting to look into. Matuidi plays a through ball to Lucas Hernández who then plays a cross at a difficult angle back to Pavard, before he hits a half volley. While the half volley itself requires lots of technique (and luck), I want to answer the following about the passes:
1. Is there a way to quantify the difficulty of the two passes that lead up the goal?
2. If yes, which pass was more difficult to make?

Fortunately, there are lots of data related to passes in the ``statsbombpy`` API which we can leverage to adopt a data-driven approach to answer our questions. The plan is to build a model that can tell us give us a pass difficulty score $p_\text{pass}$ for a pass from point $(x_1, y_1)$ to $(x_2, y_2)$ on the pitch. 

# Getting the data


In [2]:
wc2018_matches_df = sb.matches(competition_id=43, season_id=3).sort_values(
    "match_date", ascending=False
)
wc2018_matches = pl.from_pandas(wc2018_matches_df)

In [9]:
wc2018_matches

match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
i64,str,str,str,str,str,str,i64,i64,str,str,str,str,i64,str,str,str,str,str,str,str,str
8658,"""2018-07-15""","""17:00:00.000""","""International - FIFA World Cup""","""2018""","""France""","""Croatia""",4,2,"""available""","""scheduled""","""2023-07-24T13:03:36.244040""","""2021-06-13T16:17:31.694""",7,"""Final""","""Stadion Luzhniki""","""Néstor Fabián Pitana""","""Didier Deschamps""","""Zlatko Dalić""","""1.0.2""",null,null
8657,"""2018-07-14""","""16:00:00.000""","""International - FIFA World Cup""","""2018""","""Belgium""","""England""",2,0,"""available""","""scheduled""","""2023-07-24T13:02:46.901265""","""2021-06-13T16:17:31.694""",7,"""3rd Place Final""","""Saint-Petersburg Stadium""","""Alireza Faghani""","""Roberto Martínez Montoliú""","""Gareth Southgate""","""1.0.2""",null,null
8656,"""2018-07-11""","""20:00:00.000""","""International - FIFA World Cup""","""2018""","""Croatia""","""England""",2,1,"""available""","""scheduled""","""2023-07-24T13:01:34.707926""","""2021-06-13T16:17:31.694""",6,"""Semi-finals""","""Stadion Luzhniki""","""Cüneyt Çakιr""","""Zlatko Dalić""","""Gareth Southgate""","""1.0.2""",null,null
8655,"""2018-07-10""","""20:00:00.000""","""International - FIFA World Cup""","""2018""","""France""","""Belgium""",1,0,"""available""","""scheduled""","""2023-07-24T13:00:27.503953""","""2021-06-13T16:17:31.694""",6,"""Semi-finals""","""Saint-Petersburg Stadium""","""Andrés Cunha""","""Didier Deschamps""","""Roberto Martínez Montoliú""","""1.0.2""",null,null
8651,"""2018-07-07""","""16:00:00.000""","""International - FIFA World Cup""","""2018""","""Sweden""","""England""",0,2,"""available""","""scheduled""","""2023-07-24T13:08:32.098385""","""2021-06-13T16:17:31.694""",5,"""Quarter-finals""",""" Solidarnost Arena""","""Björn Kuipers""","""Jan Olof Andersson""","""Gareth Southgate""","""1.0.2""",null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
7529,"""2018-06-16""","""21:00:00.000""","""International - FIFA World Cup""","""2018""","""Croatia""","""Nigeria""",2,0,"""available""","""scheduled""","""2023-07-24T12:57:10.686569""","""2021-06-13T16:17:31.694""",1,"""Group Stage""","""Stadion Kaliningrad""","""Sandro Ricci""","""Zlatko Dalić""","""Gernot Rohr""","""1.0.2""",null,null
7578,"""2018-06-15""","""14:00:00.000""","""International - FIFA World Cup""","""2018""","""Egypt""","""Uruguay""",0,1,"""available""","""scheduled""","""2021-12-06T22:24:47.980215""","""2021-06-13T16:17:31.694""",1,"""Group Stage""",""" Ekaterinburg Arena""","""Björn Kuipers""","""Héctor Raúl Cúper""","""Óscar Washington Tabárez Silva""","""1.0.2""",null,null
7577,"""2018-06-15""","""17:00:00.000""","""International - FIFA World Cup""","""2018""","""Morocco""","""Iran""",0,1,"""available""","""scheduled""","""2023-07-24T12:54:22.880193""","""2021-06-13T16:17:31.694""",1,"""Group Stage""","""Saint-Petersburg Stadium""","""Cüneyt Çakιr""","""Hervé Renard""","""Carlos Manuel Brito Leal Queir…","""1.0.2""",null,null


In [40]:
goat = passes.select(pl.col("duration"), pl.col("location"), pl.col("pass_end_location"), pl.col("pass_angle"), pl.col("pass_length"), pl.col("pass_height"), pl.col("pass_outcome"))
goat

duration,location,pass_end_location,pass_angle,pass_length,pass_height,pass_outcome
f64,list[f64],list[f64],f64,f64,str,str
0.883,"[60.0, 40.0]","[47.0, 40.0]",3.1415927,13.0,"""Ground Pass""",null
1.68,"[49.0, 36.0]","[34.0, 20.0]",-2.323948,21.931713,"""Ground Pass""",null
2.24,"[33.0, 18.0]","[70.0, 4.0]",-0.361729,39.56008,"""High Pass""",null
1.68,"[55.0, 80.0]","[32.0, 78.0]",-3.054854,23.086792,"""Low Pass""",null
0.8,"[33.0, 72.0]","[41.0, 79.0]",0.71883,10.630146,"""Ground Pass""",null
…,…,…,…,…,…,…
2.093,"[90.0, 19.0]","[107.0, 45.0]",0.991722,31.06445,"""High Pass""","""Incomplete"""
2.0,"[91.0, 78.0]","[108.0, 78.0]",0.0,17.0,"""Ground Pass""",null
2.133,"[112.0, 76.0]","[117.0, 43.0]",-1.420425,33.37664,"""High Pass""","""Incomplete"""
